In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,244 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [983 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:13 https://ppa.launchpadcont

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
home_avg=spark.sql("""
          SELECT date,
          ROUND(AVG(price),2)
          as AVG_Four_Bedroom
          FROM home
          WHERE bedrooms = 4
          GROUP BY date
          """)
home_avg.show()


+----------+----------------+
|      date|AVG_Four_Bedroom|
+----------+----------------+
|2020-04-13|        303153.3|
|2020-02-26|        295492.0|
|2019-08-22|        331777.3|
|2019-08-08|       305486.56|
|2021-11-03|       279481.33|
|2019-08-23|       273917.83|
|2020-06-24|       363142.22|
|2019-09-29|        286234.2|
|2021-12-23|        263829.0|
|2019-08-31|       367088.29|
|2019-04-25|       240981.75|
|2020-06-08|        269630.0|
|2020-11-12|       314268.22|
|2021-04-06|       250681.71|
|2022-05-17|       279781.33|
|2020-09-12|       265752.09|
|2022-03-30|        273248.0|
|2019-01-17|       333439.09|
|2021-08-30|        363022.0|
|2019-04-26|       318184.93|
+----------+----------------+
only showing top 20 rows



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
home_avg=spark.sql("""
          SELECT date,
          ROUND(AVG(price),2)
          as AVG_Three_Beds_Baths
          FROM home
          WHERE bedrooms = 3 AND bathrooms = 3
          GROUP BY date
          """)
home_avg.show()

+----------+--------------------+
|      date|AVG_Three_Beds_Baths|
+----------+--------------------+
|2019-08-23|            304089.5|
|2021-11-03|            370813.8|
|2020-04-13|            298782.0|
|2020-02-26|           237994.75|
|2019-08-22|            288818.5|
|2019-08-08|            380345.2|
|2021-12-23|           588392.75|
|2019-04-25|            316485.6|
|2019-09-29|           280435.13|
|2020-06-24|            271525.5|
|2019-08-31|            255717.0|
|2020-09-12|            449314.0|
|2020-06-08|            251318.4|
|2021-04-06|           506720.67|
|2022-05-17|           301401.25|
|2020-11-12|            143575.5|
|2021-10-25|           329296.14|
|2020-06-22|           249791.25|
|2019-04-26|            289360.5|
|2020-01-05|           296966.71|
+----------+--------------------+
only showing top 20 rows



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

home_avg=spark.sql("""
          SELECT date, ROUND(AVG(price),2) as AVG_Three_Beds_Baths_Two_Floors
          FROM home
          WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2
          GROUP BY date
          """)
home_avg.show()

+----------+-------------------------------+
|      date|AVG_Three_Beds_Baths_Two_Floors|
+----------+-------------------------------+
|2021-11-03|                      420228.67|
|2020-04-13|                       338753.5|
|2020-02-26|                       189833.5|
|2019-08-08|                       437130.0|
|2019-08-22|                       175293.5|
|2019-08-23|                       409256.0|
|2019-09-29|                       188812.0|
|2019-04-25|                       300957.0|
|2019-08-31|                       255717.0|
|2020-06-24|                       345325.0|
|2021-12-23|                       287120.0|
|2020-06-08|                       238142.5|
|2020-09-12|                       430663.0|
|2022-05-17|                       351764.0|
|2021-10-25|                      344264.67|
|2019-04-26|                       289360.5|
|2020-01-05|                      306027.33|
|2022-03-30|                       223058.0|
|2019-01-17|                       286324.6|
|2021-02-1

In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

home_avg=spark.sql("""
          SELECT view, ROUND(AVG(price),2) as AVG_Price_View
          FROM home
          WHERE price >= 350000
          GROUP BY view
          """)
home_avg.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|AVG_Price_View|
+----+--------------+
|   7|     403005.77|
|  51|     788128.21|
|  15|      404673.3|
|  54|     798684.82|
|  11|     399548.12|
|  29|     397771.65|
|  69|     750537.94|
|  42|      396964.5|
|  87|     1072285.2|
|  73|     752861.18|
|  64|     767036.67|
|   3|      398867.6|
|  30|      397862.0|
|  34|     401419.75|
|  59|      791453.0|
|   8|     398592.71|
|  28|     402124.62|
|  22|     402022.68|
|  85|    1056336.74|
|  16|     399586.53|
+----+--------------+
only showing top 20 rows

--- 0.8769650459289551 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view,
ROUND(AVG(price),2)
as AVG_Price_View
FROM home
WHERE price >= 350000
GROUP BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------+
|view|AVG_Price_View|
+----+--------------+
|   7|     403005.77|
|  51|     788128.21|
|  15|      404673.3|
|  54|     798684.82|
|  11|     399548.12|
|  29|     397771.65|
|  69|     750537.94|
|  42|      396964.5|
|  87|     1072285.2|
|  73|     752861.18|
|  64|     767036.67|
|   3|      398867.6|
|  30|      397862.0|
|  34|     401419.75|
|  59|      791453.0|
|   8|     398592.71|
|  28|     402124.62|
|  22|     402022.68|
|  85|    1056336.74|
|  16|     399586.53|
+----+--------------+
only showing top 20 rows

--- 0.5778729915618896 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.parquet('date_built',mode='overwrite')

In [15]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('date_built')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('date_built')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
SELECT view,
ROUND(AVG(price),2)
as AVG_Price_View
FROM home
WHERE price >= 350000
GROUP BY view
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|AVG_Price_View|
+----+--------------+
|   7|     403005.77|
|  51|     788128.21|
|  15|      404673.3|
|  54|     798684.82|
|  11|     399548.12|
|  29|     397771.65|
|  69|     750537.94|
|  42|      396964.5|
|  87|     1072285.2|
|  73|     752861.18|
|  64|     767036.67|
|   3|      398867.6|
|  30|      397862.0|
|  34|     401419.75|
|  59|      791453.0|
|   8|     398592.71|
|  28|     402124.62|
|  22|     402022.68|
|  85|    1056336.74|
|  16|     399586.53|
+----+--------------+
only showing top 20 rows

--- 0.6242365837097168 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home")


False